# WBC Segmentation (Unstained) - Kaggle GPU Notebook

Run this end-to-end in a Kaggle Notebook with GPU. Adjust the dataset path as noted below.

In [ ]:
# Clone the repo if not already present (Kaggle starts in /kaggle/working)
import os, subprocess
from pathlib import Path

REPO_URL = "https://github.com/mpotalib/dip-blood_cell_segmentations.git"
REPO_DIR = Path("/kaggle/working/dip-blood_cell_segmentations")

if not REPO_DIR.exists():
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
os.chdir(REPO_DIR)
print("CWD:", Path.cwd())

In [ ]:
# Install dependencies
!pip install -r requirements.txt

In [ ]:
# Point to your Kaggle dataset containing data/train|val|test with images + masks (or annotations)
# Example: upload a dataset and set DATASET_BASE to /kaggle/input/your-dataset-name
from pathlib import Path
import os, shutil

DATASET_BASE = Path("/kaggle/input/dip-wbc-dataset")  # TODO: set to your dataset name
TARGET = Path("data")

if not DATASET_BASE.exists():
    raise FileNotFoundError(f"Dataset path not found: {DATASET_BASE}. Update DATASET_BASE above.")

# If data is already structured as data/train/images etc, just symlink
if TARGET.exists():
    if TARGET.is_symlink():
        TARGET.unlink()
    else:
        shutil.rmtree(TARGET)
os.symlink(DATASET_BASE, TARGET)
print("Linked", DATASET_BASE, "->", TARGET)

# If masks are not pre-generated but annotations exist, create masks
for split in ["train", "val", "test"]:
    img_dir = TARGET / split / "images"
    ann_dir = TARGET / split / "annotations"
    mask_dir = TARGET / split / "masks"
    if img_dir.exists() and ann_dir.exists() and not mask_dir.exists():
        mask_dir.mkdir(parents=True, exist_ok=True)
        !python prepare_masks.py --images-dir {img_dir} --annotations-dir {ann_dir} --output-dir {mask_dir}
    else:
        print(f"Split {split}: images={img_dir.exists()}, annotations={ann_dir.exists()}, masks={mask_dir.exists()}")

In [ ]:
# Train (choose config: baseline or deeplab)
!python train.py --config experiments/deeplab.yaml


In [ ]:
# Evaluate on val/test and export qualitative masks for the report/deck
!python evaluate.py --config experiments/deeplab.yaml --checkpoint outputs/deeplab/checkpoints/best.pt --split val --save-dir outputs/deeplab/preds_val --limit 20
!python evaluate.py --config experiments/deeplab.yaml --checkpoint outputs/deeplab/checkpoints/best.pt --split test --save-dir outputs/deeplab/preds_test --limit 20


## Train with log and plot curves
Use tee to capture stdout to train.log, then parse and plot train/val curves.


In [ ]:
# Train DeepLab and save log for plotting
!python train.py --config experiments/deeplab.yaml | tee train.log


In [ ]:
# Parse train.log and plot loss/Dice curves
import re, matplotlib.pyplot as plt
train_loss, val_loss, val_dice = [], [], []
with open('train.log') as f:
    for line in f:
        m = re.search(r'Epoch (\d+)/(\d+).*train loss ([0-9.]+)', line)
        if m:
            train_loss.append(float(m.group(3)))
        m = re.search(r'Validation - loss: ([0-9.]+) \| dice: ([0-9.]+)', line)
        if m:
            val_loss.append(float(m.group(1)))
            val_dice.append(float(m.group(2)))
if train_loss and val_loss:
    plt.figure(figsize=(8,4))
    plt.plot(train_loss, label='train loss')
    plt.plot(val_loss, label='val loss')
    plt.xlabel('epoch'); plt.ylabel('loss'); plt.legend(); plt.tight_layout()
if val_dice:
    plt.figure(figsize=(6,4))
    plt.plot(val_dice, label='val Dice')
    plt.xlabel('epoch'); plt.ylabel('Dice'); plt.legend(); plt.tight_layout()
plt.show()
